## Introdução
O módulo principal do projeto `minimize.py` contém duas classes:
- `Minimizer` implementa e itera os algoritmos de minimização irrestrita
- `Drawer` faz visualizações gráficas de algoritmos de minimização de funções $\mathbb{R}^2 \Rightarrow \mathbb{R}$ com o auxílio da biblioteca [plotly](https://plot.ly/python/)

In [2]:
import minimize as mini

## Secção 1: Classe de minimização
- $f$, $f_{grad}$ e $f_{hess}$ são funções pré-definidas passadas para a classe
- `inputs` é a dimensão do domínio
- `x_0` é o ponto inicial do algoritmo
- o vetor `x` armazena os passos das iterações
- `B` é o vetor das matrizes de aproximação da Hessiana, no caso do algoritmo quase-Newton
- `alpha` é a constante pré-estabelecida para a condição de Armijo (a literatura sugere que os valores ótimos para essa constante costumam ser próximos do _default_ estabelecido de `0.1`)
```python
class Minimizer:
    def __init__(self, f, inputs, x_0 = np.array([0,0]), alpha = 0.1):
        self.f = f
        self.inputs = inputs
        self.f_grad = lambda inp: inp
        self.f_hess = lambda inp: np.identity(inputs)
        self.x = [x_0]
        self.steps = []
        self.alpha = alpha
        self.B = []
```

### Busca Linear
Usa _backtracking_ a partir de $t = 1$ com fator $0.5$ enquanto a condição de Armijo
$ f(x+t\cdot d) < f(x) + \alpha t \nabla f(x) \cdot d$ for falsa.
Se $t = 0$, muitos passos sem sucesso foram executados e a busca linear falhou.
```python
    def line_search(self, dr):
        t = 1
        x = self.x[-1]
        #Armijo condition
        while self.f(x + t*dr) >= self.f(x) + self.alpha*t*np.inner(self.f_grad(x), dr):
            if t == 0:
                print(dr, self.f(x), self.alpha*t*np.inner(self.f_grad(x), dr))
                print('line_search failed')
                return -1
            t /= 2
        return t
 ```

### Passo Gradiente
```python
    def gradient_step(self):
        x = self.x[-1]
        direction = -self.f_grad(x) # toma a direção como -grad_f(x)
        t = self.line_search(direction) # busca linear
        self.steps.append(t*direction)
        self.x.append(self.x[-1]+self.steps[-1]) # adiciona o x obtido na lista da classe
        return t*direction
 ```

### Passo de Newton

- Resolve sistema linear $\nabla ^2f(x_k) \cdot d_k = -\nabla f(x_k)$ e usa $d_k$ como passo
```python
def newton_step(self):
        x = self.x[-1]
        step = np.linalg.solve(self.f_hess(x), -self.f_grad(x))
        self.steps.append(step)
        self.x.append(self.x[-1]+self.steps[-1])
        return step
```

### Passo BFGS
 - Resolve $B_k \cdot x = -\nabla f(x_k)$ para determinar direção
 - Faz busca linear na direção determinada
 - Define $x_{k+1}$
 - Define $B_{k+1}$ pela fórmula do algoritmo BFGS. Sejam $\mathbf{s_k}$ o passo determinado pela busca linear e $\mathbf{y}_k = {\nabla f(\mathbf{x}_{k+1}) - \nabla f(\mathbf{x}_k)}$,
 $$B_{k+1} = B_k + \frac{\mathbf{y}_k \mathbf{y}_k^{\mathrm{T}}}{\mathbf{y}_k^{\mathrm{T}} \mathbf{s}_k} - \frac{B_k \mathbf{s}_k \mathbf{s}_k^{\mathrm{T}} B_k^{\mathrm{T}} }{\mathbf{s}_k^{\mathrm{T}} B_k \mathbf{s}_k}$$
```python
    def BFGS_step(self):
        x = self.x[-1]
        direction = np.linalg.solve(self.B[-1], -self.f_grad(x))
        t = self.line_search(direction)
        if t == -1:
            return -1
        step = t*direction
        self.steps.append(step)
        self.x.append(self.x[-1]+self.steps[-1])
        y = self.f_grad(self.x[-1]) - self.f_grad(self.x[-2])
        B_ = self.B[-1]
        B_0 = np.outer(y,y)/np.inner(y, step)
        B_1 = - np.outer(B_ @ step, B_ @ step)/(np.transpose(step) @ B_ @ step)
        #computes next Hessian approximation
        self.B.append(B_ + B_0 + B_1)
        return step
 ```

### Determinação do passo
A função `step` realiza a próxima iteração no algoritmo de minimização. Pode ser passado um argumento `method` que determina qual método deve ser usado.
Caso `method` tenha o valor `'default'`, verificamos se é possível aplicar o método de Newton. Como critério padrão, se a Hessiana for positivo-definida (testada por fatoração Cholesky) utiliza-se o método de Newton, caso contrário usamos o método do gradiente.
```python
    def step(self, method = 'default'):
        if method == 'default' and self.f_hess != None:
            if np.array_equal(self.f_hess(self.x[-1]), np.transpose(self.f_hess(self.x[-1]))):
                try:
                    np.linalg.cholesky(self.f_hess(self.x[-1]))
                    method = 'newton'
                except np.linalg.LinAlgError:
                    print('non-positive definite matrix')
        if method == 'BFGS':
            if len(self.B) == 0:
                self.B.append(np.identity(self.inputs))
            self.BFGS_step()
        elif method == 'newton':
            self.newton_step()
        else: # gradient descent is default
            self.gradient_step()
        return (self.steps[-1], method)
```

### Iterações
O método `iterate` automatiza o processo de iteração do algoritmo.
Os critérios de parada possíveis tem uso e valores dependendo do que é passado à função:
 - $n$ define o máximo de iterações
 - Se `delta` está definido, o algoritmo para quando $||\nabla f(x_k)|| < \delta$
```python
    def iterate(self, n=2000, delta=10e-4, method='default', log=False, delta_f = None):
        for _ in range(n):
            #... código omitido ..
            if delta != None and np.linalg.norm(self.f_grad(self.x[-1])) < delta:
                break
            st = self.step(method=method)
            print(st)
 ```

## Classe gráfica
A implementação da classe `Drawer` não será detalhada nesse caderno. Apenas uma apresentação da assinatura e breve descrição dos principais métodos devem ser suficientes para acompanhar os exemplos práticos seguintes.
```python
class Drawer:
    #... código omitido ..
    def draw_f(self, vet_f, mim, name = 'function'): # Desenha as superfícies
        # ...
    def draw_curve(self, vet_f, mim, X_0, X_f, color='darkblue'): # Marca uma dispersão de pontos
        # ...
    def draw_path(self, vet_f, mim, path, color='darkblue', projection=False, density = 20, name = 'algorithm'): 
        # Desenha um caminho baseado nas iterações do minimizador
        # ...
    def draw_marker(self, vet_f, pos, color='black', symbol_type='square', name='marker'): # Destaca uma posição especificada
        # ...
    def show(self): # Reenderiza os gráficos
        # ...
```

## Secção 2: Testes
Para testar a performance geral dos algoritmos implementados vamos rodar alguns exemplos de funções teste padrão para problemas de minimização irrestrita N-dimensional. Boa parte dos seguintes exemplos foram tirados do artigo [_Test Functions for Optimization_](https://en.wikipedia.org/wiki/Test_functions_for_optimization) da Wikipedia. As funções teste foram adicionadas no arquivo `test_functions.py`.

O cálculo do gradiente e da Hessiana foi feito manualmente de forma analítica para as funções mais simples. Para funções mais complicadas, no entanto, esse processo fica excessivamente trabalhoso, então foi implementada uma função que aproxima o valor do gradiente usando diferenciação numérica combinada com funções _lambda_, próprias de programação funcional. Infelizmente, nesses problemas fica difícil computar a Hessiana de forma exata, então devemos nos satisfazer com a aplicação do método BFGS no lugar de Newton.
``` python
def aprox_deriv(f, x, h=10**(-4)):  # symetric difference quotient
    dy = f(x+h) - f(x-h)
    return dy/(2*h)

def aprox_grad(f, x):
    grad = []
    for i in range(len(x)):
        fs = lambda inp: f([*x[0:i], inp, *x[(i+1):len(x)]])
        grad.append(aprox_deriv(fs, x[i]))
    return np.array(grad)
```

Os testes a seguir iteram sobre todas as funções definidas em `test_functions.py` com valor inicial aleatório dentro do intervalo especificado e com um número variável de dimensões para o problema. O valor final de `x` e a velocidade de convergência dependem fortemente das condições iniciais e o algoritmo utilizado.

In [3]:
import numpy as np
import math
import pandas as pd
import sys
import minimize as mini
import test_functions
from test_functions import fns
from IPython.display import display

delta = 10e-4
method = "BFGS"

print('method:',method)

for item in fns:
    for n in range(2, 5):
        fn = item(n)
        if fn == None:  # Verifies if n is a valid dimension for the problem
            break
        if 'name' in fn:
            print(n, '-D, ', fn['name'], sep='')
        if fn['search_region'] == None:
            fn['search_region'] = [-500, 500]
        x_0 = np.random.uniform(fn['search_region'][0], fn['search_region'][1], n)
        print('x_0 =', x_0)
        mim = mini.Minimizer(fn['function'], n, x_0)
        mim.f_grad = fn['gradient']
        mim.f_hess = fn['hessian']
        mim.iterate(method=method, log=True, n=10000)
        display(mim.pretty_print())
        print("minimo global:", fn['global_minimum'])
        print()

method: BFGS
2-D, Rastrigin
x_0 = [-385.52041677  417.60336719]


,x,f(x),grad_f(x),method
0,"[-385.52, 417.603]",323056.446,"[-763.003, 797.208]",BFGS
1,"[-4.019, 18.999]",377.200,"[-15.546, 37.748]",BFGS
2,"[-1.383, -6.027]",55.786,"[-45.037, -22.79]",BFGS
3,"[-0.915, -6.138]",43.445,"[30.03, -60.272]",BFGS
4,"[1.148, -1.867]",12.136,"[52.747, 42.867]",BFGS
5,"[1.056, -2.024]",5.922,"[23.618, -13.292]",BFGS
6,"[0.986, -1.988]",4.992,"[-3.629, 0.645]",BFGS
7,"[0.995, -1.99]",4.975,"[0.118, 0.126]",BFGS
8,"[0.995, -1.99]",4.975,"[-0.002, -0.017]",BFGS
9,"[0.995, -1.99]",4.975,"[-0.0, 0.0]",BFGS


minimo global: [0, 0]

3-D, Rastrigin
x_0 = [ 85.11559455 133.87088498 423.23555474]
line_search failed:
x = 6.211714270958129 16.84102768766591 7.541815149848369
grad_x = [ 73.44607506 -19.15013545  -1.23506425]
f(x) = 410.9463177836362
line_search failed:
x = 6.211714270958129 16.84102768766591 7.541815149848369
grad_x = [ 73.44607506 -19.15013545  -1.23506425]
f(x) = 410.9463177836362
line_search failed:
x = 6.211714270958129 16.84102768766591 7.541815149848369
grad_x = [ 73.44607506 -19.15013545  -1.23506425]
f(x) = 410.9463177836362
line_search failed:
x = 6.211714270958129 16.84102768766591 7.541815149848369
grad_x = [ 73.44607506 -19.15013545  -1.23506425]
f(x) = 410.9463177836362
line_search failed:
x = 6.211714270958129 16.84102768766591 7.541815149848369
grad_x = [ 73.44607506 -19.15013545  -1.23506425]
f(x) = 410.9463177836362
line_search failed:
x = 6.211714270958129 16.84102768766591 7.541815149848369
grad_x = [ 73.44607506 -19.15013545  -1.23506425]
f(x) = 410.94631778363

line_search failed:
x = 6.211714270958129 16.84102768766591 7.541815149848369
grad_x = [ 73.44607506 -19.15013545  -1.23506425]
f(x) = 410.9463177836362
line_search failed:
x = 6.211714270958129 16.84102768766591 7.541815149848369
grad_x = [ 73.44607506 -19.15013545  -1.23506425]
f(x) = 410.9463177836362
line_search failed:
x = 6.211714270958129 16.84102768766591 7.541815149848369
grad_x = [ 73.44607506 -19.15013545  -1.23506425]
f(x) = 410.9463177836362
line_search failed:
x = 6.211714270958129 16.84102768766591 7.541815149848369
grad_x = [ 73.44607506 -19.15013545  -1.23506425]
f(x) = 410.9463177836362
line_search failed:
x = 6.211714270958129 16.84102768766591 7.541815149848369
grad_x = [ 73.44607506 -19.15013545  -1.23506425]
f(x) = 410.9463177836362
line_search failed:
x = 6.211714270958129 16.84102768766591 7.541815149848369
grad_x = [ 73.44607506 -19.15013545  -1.23506425]
f(x) = 410.9463177836362
line_search failed:
x = 6.211714270958129 16.84102768766591 7.541815149848369
grad

line_search failed:
x = 6.211714270958129 16.84102768766591 7.541815149848369
grad_x = [ 73.44607506 -19.15013545  -1.23506425]
f(x) = 410.9463177836362
line_search failed:
x = 6.211714270958129 16.84102768766591 7.541815149848369
grad_x = [ 73.44607506 -19.15013545  -1.23506425]
f(x) = 410.9463177836362
line_search failed:
x = 6.211714270958129 16.84102768766591 7.541815149848369
grad_x = [ 73.44607506 -19.15013545  -1.23506425]
f(x) = 410.9463177836362
line_search failed:
x = 6.211714270958129 16.84102768766591 7.541815149848369
grad_x = [ 73.44607506 -19.15013545  -1.23506425]
f(x) = 410.9463177836362
line_search failed:
x = 6.211714270958129 16.84102768766591 7.541815149848369
grad_x = [ 73.44607506 -19.15013545  -1.23506425]
f(x) = 410.9463177836362
line_search failed:
x = 6.211714270958129 16.84102768766591 7.541815149848369
grad_x = [ 73.44607506 -19.15013545  -1.23506425]
f(x) = 410.9463177836362
line_search failed:
x = 6.211714270958129 16.84102768766591 7.541815149848369
grad

...

minimo global: [0, 0]

4-D, Sphere
x_0 = [-2.74981609e+02 -2.70026618e+02 -4.70575731e+02 -1.19557216e-01]


,x,f(x),grad_f(x),method
0,"[-274.982, -270.027, -470.576, -0.12]",369970.793,"[-549.963, -540.053, -941.151, -0.239]",BFGS
1,"[0.0, 0.0, 0.0, 0.0]",0.000,"[0.0, 0.0, 0.0, 0.0]",BFGS


minimo global: [0, 0]

2-D, Goldstein price
x_0 = [-0.86819525  0.83857193]


,x,f(x),grad_f(x),method
0,"[-0.868, 0.839]",49487.100,"[-12509.725, 174881.747]",BFGS
1,"[-0.773, -0.496]",120.146,"[-736.161, -690.753]",BFGS
2,"[-0.392, -0.47]",54.724,"[317.119, 262.761]",BFGS
3,"[-0.506, -0.478]",32.371,"[51.375, 13.328]",BFGS
4,"[-0.528, -0.479]",31.839,"[-0.45, -37.56]",BFGS
5,"[-0.527, -0.478]",31.829,"[1.914, -35.192]",BFGS
6,"[-0.526, -0.476]",31.766,"[10.924, -26.046]",BFGS
7,"[-0.526, -0.47]",31.636,"[21.977, -14.478]",BFGS
8,"[-0.535, -0.453]",31.289,"[39.174, 5.58]",BFGS
9,"[-0.561, -0.418]",30.724,"[51.543, 31.483]",BFGS


minimo global: [0, -1]

